In [1]:
import pandas as pd
import numpy as np
from models.gan import GAN

In [2]:
df_pokemon= pd.read_csv("data/Pokemon.csv")
df_pokemon = df_pokemon.drop(columns=['Name', 'Total'], axis=1)
df_pokemon.head()

,#,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Grass,Poison,80,82,83,100,100,80,1,False
3,3,Grass,Poison,80,100,123,122,120,80,1,False
4,4,Fire,NaN,39,52,43,60,50,65,1,False


In [4]:
noise = np.random.normal(0, 1, df_pokemon.shape) 
gan = GAN(data=df_pokemon, noise=noise, epochs=2)
generator = gan.create_generator()
discriminator = gan.create_discriminator()
gan_model = gan.compile(generator=generator, discriminator=discriminator)
trained_gan = gan.train(generator=generator,discriminator=discriminator, gan=gan)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 907us/step


TypeError: Expected binary or unicode string, got 1